In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/부트캠프/train_common_v2.2_1104.csv')
test = pd.read_csv('/content/drive/MyDrive/부트캠프/test_common_v2.2_1104.csv')
target =  pd.read_csv('/content/drive/MyDrive/부트캠프/store_train.csv')
submit = pd.read_csv('/content/drive/MyDrive/부트캠프/store_submission.csv')

In [3]:
train_ft = train.iloc[:,1:].copy()
test_ft = test.iloc[:,1:].copy()

In [4]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [5]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류_수정,6
주구매_중분류_수정,19
대분류_수정_평균금액최대,6


In [6]:
from sklearn.preprocessing import OneHotEncoder

cols = train_ft.select_dtypes(include=['object']).columns
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(train_ft[cols])

tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft, tmp], axis=1).drop(columns=cols)

tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft, tmp], axis=1).drop(columns=cols)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_중분류_수정_의류_기타,주구매_중분류_수정_의류_의류,주구매_중분류_수정_의류_잡화,주구매_중분류_수정_의류_화장품,대분류_수정_평균금액최대_가정용품,대분류_수정_평균금액최대_명품,대분류_수정_평균금액최대_생식품,대분류_수정_평균금액최대_아동,대분류_수정_평균금액최대_영,대분류_수정_평균금액최대_의류
0,-0.369867,0.002987,0.257728,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.04255,1.704365,-0.252586,-0.328478,2.466562,-0.463393,-0.151451,-0.297664,-0.362477,-0.926895
1,0.144110,-0.356452,-1.008554,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.04255,-0.586729,-0.252586,-0.328478,-0.405423,2.157996,-0.151451,-0.297664,-0.362477,-0.926895
2,1.943028,-0.869935,0.036742,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.04255,1.704365,-0.252586,-0.328478,-0.405423,2.157996,-0.151451,-0.297664,-0.362477,-0.926895
3,3.793345,-1.023980,-0.080558,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.04255,1.704365,-0.252586,-0.328478,2.466562,-0.463393,-0.151451,-0.297664,-0.362477,-0.926895
4,0.452496,-0.613193,0.302875,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.04255,-0.586729,-0.252586,-0.328478,-0.405423,2.157996,-0.151451,-0.297664,-0.362477,-0.926895


In [8]:
target = target["target"]

# XGBoost

In [9]:
import pandas as pd
from xgboost import XGBClassifier


model = XGBClassifier(random_state=42)
model.fit(train_ft, target)

# 피처 중요도 DataFrame 생성
importance_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Importance': model.feature_importances_
})

# 모든 피처를 출력할 수 있도록 pandas 출력 옵션 설정
pd.set_option('display.max_rows', None)  # 모든 행을 출력
pd.set_option('display.max_columns', None)  # 모든 열을 출력

# 중요도를 기준으로 정렬하여 모든 피처 출력
importance_XGboost = importance_df.sort_values(by='Importance', ascending=False)
importance_XGboost

,Feature,Importance
12,저녁_구매비율,0.036359
117,대분류_수정_생식품_구매금액,0.035765
14,아침_구매건수,0.022190
177,주구매_대분류_수정_아동,0.015223
148,pivot_cnt_아동_완구,0.015129
191,주구매_중분류_수정_아동_완구,0.015028
196,주구매_중분류_수정_의류_의류,0.014878
203,대분류_수정_평균금액최대_영,0.013551
186,주구매_중분류_수정_명품_명품,0.012260
194,주구매_중분류_수정_영_의류,0.011175


In [10]:
importance_XGboost.to_csv('/content/drive/MyDrive/부트캠프/XGboost_importance.csv', index=False)

# LightGBM

In [12]:
import pandas as pd
from lightgbm import LGBMClassifier

# LightGBM 모델 학습
model = LGBMClassifier(random_state=42)
model.fit(train_ft, target)

# 피처 중요도 DataFrame 생성
importance_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Importance': model.feature_importances_
})

# 모든 피처를 출력할 수 있도록 pandas 출력 옵션 설정
pd.set_option('display.max_rows', None)  # 모든 행을 출력
pd.set_option('display.max_columns', None)  # 모든 열을 출력

# 중요도를 기준으로 정렬하여 모든 피처 출력
importance_LightGBM = importance_df.sort_values(by='Importance', ascending=False)
importance_LightGBM


[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.105294 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22867
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 194
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


,Feature,Importance
8,일별평균구매건수,77
12,저녁_구매비율,77
105,최소구매액,69
148,pivot_cnt_아동_완구,61
117,대분류_수정_생식품_구매금액,59
154,pivot_cnt_의류_의류,56
83,브랜드다양성_비율,52
4,여름_구매비율,52
169,대분류_수정_평균구매가격,50
2,주말방문비율,48


# Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(random_state=42, max_iter=1000)
logistic_model.fit(train_ft, target)

# Logistic Regression 피처 중요도 계산
logistic_importance_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Coefficient': logistic_model.coef_[0],  # 음양수
    'Absolute Importance': np.abs(logistic_model.coef_[0])  # 절대값
})

importance_Logistic = logistic_importance_df.sort_values(by='Absolute Importance', ascending=False)
importance_Logistic



,Feature,Coefficient,Absolute Importance
126,대분류_수정_생식품_구입비중,-0.321891,0.321891
113,저가_구매비율,0.268712,0.268712
109,저가_구매횟수,0.261933,0.261933
122,대분류_수정_명품_구매금액,-0.257656,0.257656
154,pivot_cnt_의류_의류,-0.253180,0.253180
12,저녁_구매비율,0.239851,0.239851
114,중저가_구매비율,0.235103,0.235103
10,아침_구매비율,-0.225639,0.225639
125,대분류_수정_의류_구입비중,-0.214638,0.214638
145,pivot_cnt_식료품_식료품,-0.214270,0.214270


Decision Tree

In [18]:
from sklearn.tree import DecisionTreeClassifier


dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(train_ft, target)

importance_dt_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Importance': dt_model.feature_importances_
})

# 중요도를 기준으로 정렬하여 모든 피처 출력
importance_decision_tree = importance_dt_df.sort_values(by='Importance', ascending=False)
importance_decision_tree


,Feature,Importance
12,저녁_구매비율,0.103593
117,대분류_수정_생식품_구매금액,0.059733
2,주말방문비율,0.015841
105,최소구매액,0.015378
130,A112000지점_구매금액,0.015016
18,월초_구매비율,0.014229
108,구매금액표준편차,0.013952
125,대분류_수정_의류_구입비중,0.013881
115,중고가_구매비율,0.013416
114,중저가_구매비율,0.013249


# Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_ft, target)

importance_rf_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Importance': rf_model.feature_importances_
})


importance_random_forest = importance_rf_df.sort_values(by='Importance', ascending=False)
importance_random_forest


,Feature,Importance
12,저녁_구매비율,0.035319
11,점심_구매비율,0.019789
117,대분류_수정_생식품_구매금액,0.015774
23,주_구매시간대,0.015642
16,저녁_구매건수,0.014980
14,아침_구매건수,0.014083
104,최대구매액,0.014074
10,아침_구매비율,0.014025
103,평균구매액,0.013392
169,대분류_수정_평균구매가격,0.013039


In [20]:
importance_XGboost.to_csv('/content/drive/MyDrive/부트캠프/XGboost_importance.csv', index=False)
importance_Logistic.to_csv('/content/drive/MyDrive/부트캠프/Logistic_importance.csv', index=False)
importance_decision_tree.to_csv('/content/drive/MyDrive/부트캠프/decision_tree_importance.csv', index=False)
importance_random_forest.to_csv('/content/drive/MyDrive/부트캠프/random_forest_importance.csv', index=False)

## importance 수치를 어케 봐야하는지

1. Logistic Regression
계수 해석: 각 피처의 계수(모델의 파라미터)는 피처가 타겟 클래스에 미치는 영향을 나타냅니다.
양수 계수: 해당 피처가 기혼일 확률을 증가시킴.
음수 계수: 해당 피처가 기혼일 확률을 감소시킴.
절대값: 계수의 절대값이 클수록 해당 피처가 모델 예측에 더 큰 영향을 미친다는 뜻.
피처 제거 기준:
음수 계수 중 절대값이 큰 피처는 기혼을 예측하는 데 방해가 될 수 있으며, 이 피처를 제거해 볼 수 있습니다.

2. LightGBM
중요도 해석: 피처 중요도는 주로 기여도를 나타내며, 트리 모델에서는 상대적인 크기로 중요도를 평가합니다.
중요도가 클수록 모델 성능에 긍정적인 기여를 하는 피처로 해석됩니다.
피처 제거 기준:
중요도가 낮은 피처는 모델 성능에 기여하지 않으므로 제거할 수 있습니다. 특히, 중요도가 0에 가까운 피처는 제거하는 것이 좋습니다.

3. Decision Tree
중요도 해석: 피처의 중요도는 주로 노드 분할의 빈도와 정보 이득을 기반으로 하며, 중요도가 클수록 모델 성능에 기여하는 피처입니다.
피처 제거 기준:
중요도가 낮은 피처는 모델을 간소화하고 해석 가능성을 높이기 위해 제거할 수 있습니다.

4. XGBoost
중요도 해석: XGBoost도 LightGBM과 유사하게 피처 중요도를 계산합니다. 일반적으로 Gain, Cover, Frequency 등의 메트릭으로 피처의 기여도를 평가할 수 있습니다.
Gain: 해당 피처가 모델 성능 향상에 얼마나 기여했는지를 나타냅니다.
피처 제거 기준:
Gain이 낮은 피처는 모델 성능에 크게 기여하지 않으므로 제거하는 것이 좋습니다.

5. Random Forest
중요도 해석: Random Forest는 피처가 노드 분할에 얼마나 자주 사용되었는지를 기반으로 피처 중요도를 계산합니다.
피처 제거 기준:
중요도가 낮은 피처는 제거할 수 있으며, 중요도가 0에 가까운 피처는 모델 성능을 방해하는 요소가 될 수 있습니다.